In [1]:
import ee
ee.Initialize()

In [2]:
import geepy
import math
import sys

In [3]:
config = geepy.params.configParams('input_classification_v32.json')

amostras = ee.FeatureCollection(config.params['samples']['2016v13'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

tiles = ee.FeatureCollection(config.params['lsTiles'])

pivots = {i: ee.FeatureCollection(config.params['pivots'] + i) for (i) in config.params['years2process']}

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['sr']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['sr']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['sr']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [4]:
tile = [geepy.feature.getTiles(tiles, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

In [5]:
# Modelo digital de elevação
srtm = ee.Image(config.params['srtm'])
#alos = ee.Image(config.params['alos']).select('MED')

# Localities distance
towns = ee.FeatureCollection(config.params['towns'])
rivers = ee.FeatureCollection(config.params['hidroBDGEx'])

dtown = towns.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(srtm)

#slope = ee.Terrain.slope(alos)
#aspect = ee.Terrain.aspect(alos).divide(180).multiply(math.pi).sin()
#hillshade = ee.Terrain.hillshade(alos)

ntl30m = {}
for i in config.params['years2process']:
    viirs = ee.Image(config.params['VIIRS'][i]).select('avg_rad').divide(100)
    ntl30m[i] = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )

In [6]:
#### Test water mask
wmask = driver.lt(300)

In [7]:
landsat_min = ee.Image('users/fernandompimenta/AIBA/landsat/landsat19902018v31Min')
landsat_max = ee.Image('users/fernandompimenta/AIBA/landsat/landsat19902018v31Max')

In [8]:
landsat = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']


    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) == 2018):
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, '2018-09-06').map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'
        
    lsEdge = filtered.map(geepy.image.edgeRemoval).median()
        
    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    fEdgeRemovedMin = filtered.map(geepy.image.edgeRemoval).min()
    fEdgeRemovedMax = filtered.map(geepy.image.edgeRemoval).max()

    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ntl30m[year], 'ntl')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, wmask, 'wmask')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, srtm, 'srtm')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, slope, 'slope')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, aspect, 'aspect')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, hillshade, 'hillshade')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dtown, 'dtown')
        
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('blue'), 'min_blue')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('green'), 'min_green')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('red'), 'min_red')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('nir'), 'min_nir')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('swir1'), 'min_swir1')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('swir2'), 'min_swir2')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMin.select('thermal'), 'min_thermal')
        
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('blue'), 'max_blue')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('green'), 'max_green')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('red'), 'max_red')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('nir'), 'max_nir')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('swir1'), 'max_swir1')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('swir2'), 'max_swir2')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, fEdgeRemovedMax.select('thermal'), 'max_thermal')
        
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, driver, 'driver')

    fEdgeRemoved = geepy.image.calcNDBI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcEVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcSAVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDWI_L(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDWI_WB(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcRatio(fEdgeRemoved)

    fEdgeRemoved = geepy.image.tassCapTransformation(fEdgeRemoved, satellite)

    ndvithermal = fEdgeRemoved.select('ndvi').divide(fEdgeRemoved.select('thermal'))
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ndvithermal, 'ndvithermal')
        
    landsat[year] = fEdgeRemoved.clip(watersheds)
        
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2019

In [9]:
for year in config.params['years2process']:
    landsat[year] = landsat[year].multiply(10000)

In [10]:
# To drive
tasks = {year: geepy.image.send2drive(landsat[year].select(['swir1','nir','green']).toInt(), config.params['extent'], 'landsat'+year+'v32', 'landsat_v32', 30) for year in config.params['years2process']}

In [11]:
for i in tasks.keys():
   [tasks[i].start()]

In [10]:
tasks = geepy.image.send2drive(landsat[year].select(['swir1','nir','green']).toInt(), config.params['extent'], 'landsat2019v32', 'landsat_v32', 30)

In [11]:
tasks.start()